# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [38]:
import os
import optuna
from tsai.optuna import *
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *
from optuna.distributions import *
from optuna.samplers import TPESampler

In [52]:
config = AttrDict(
    study_name = 'general', # name of the Optuna study
    study_type = 'bayesian', # 'bayesian' or 'gridsearch' or 'random'
    n_trials = 2, # number of trials
    train_nb = f'{os.getcwd()}/training.ipynb', # path to the notebook to be executed
    search_space = {
        "arch.attn_dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.d_model": IntUniformDistribution(32, 512, 32),
        "arch.n_layers": IntUniformDistribution(1, 6, 1),
        "arch.n_heads": IntUniformDistribution(1, 8, 1),
        "arch.d_ff": IntUniformDistribution(32, 512, 32),
        "init_weights": CategoricalDistribution([True, False]), # true = kaiming
    },
    # Add extra parameters that are fixed, but not part of the search space
    extra_params = {
        "n_epoch": 1,
        "bs": 128,
        "use_wandb": True,
        "wandb_group": config.study_name
    }
)

In [48]:
def create_objective(train_nb, search_space, extra_params=None):
    """
        Create objective function to be minimized by Optuna.
        Inputs:
            trial: Optuna trial object
            train_nb: path to the training notebook
            search_vars: keys of the search space to be used
            wandb_group: name of the wandb group to be used
        Output:
            valid_loss: validation loss
    """
    def objective(trial:optuna.Trial):
        # Define the parameters to be passed to the training notebook through papermill
        pm_parameters = {}
        for k,v in search_space.items():
            pm_parameters['config.' + k] = trial._suggest(k, v)

        # Add the extra parameters to the dictionary. The key of every parameter 
        # must be 'config.<param_name>'
        if extra_params is not None:
            for k,v in extra_params.items():
                pm_parameters['config.' + k] = v

        # Call the training notebook using papermill (don't print the output)
        stdout_file = open('tmp/pm_stdout.txt', 'w')
        stderr_file = open('tmp/pm_stderr.txt', 'w')

        pm.execute_notebook(
            train_nb,
            './tmp/pm_output.ipynb',
            parameters = pm_parameters,
            stdout_file = stdout_file,
            stderr_file = stderr_file
        )

        # Close the output files
        stdout_file.close()
        stderr_file.close()

        # Get the output value of interest from the source notebook
        %store -r valid_loss
        return valid_loss

    return objective

In [60]:
obj = create_objective(config.train_nb, search_space, extra_params=extra_params)
study = run_optuna_study(obj, study_type='bayesian', direction='minimize', path='./tmp',
                 study_name=config.study_name, n_trials=config.n_trials)

[I 2023-03-24 21:12:25,656] A new study created in memory with name: general
  0%|          | 0/2 [05:56<?, ?it/s]

[W 2023-03-24 21:18:22,278] Trial 0 failed with parameters: {'arch.attn_dropout': 0.0, 'arch.dropout': 0.2, 'arch.d_model': 480, 'arch.n_layers': 3, 'arch.n_heads': 6, 'arch.d_ff': 320, 'init_weights': True} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/pip-global/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_15494/3333125453.py", line 28, in objective
    pm.execute_notebook(
  File "/usr/local/pip-global/papermill/execute.py", line 113, in execute_notebook
    nb = papermill_engines.execute_notebook_with_engine(
  File "/usr/local/pip-global/papermill/engines.py", line 49, in execute_notebook_with_engine
    return self.get_engine(engine_name).execute_notebook(nb, kernel_name, **kwargs)
  File "/usr/local/pip-global/papermill/engines.py", line 367, in execute_notebook
    cls.execute_managed_notebook(nb_man, kernel_name, log_output=log_output, **kwargs)
  File "